# Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import contextily as cx
from branca.colormap import StepColormap
import matplotlib.pyplot as plt
import folium
from folium import plugins
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Load data

## Waterdiertjes dataset

In [ ]:
df = pd.read_csv("../.data/in/waterdiertjes_cleaned.csv")

Jaar als kolom toevoegen

In [ ]:
df['Datum2'] = pd.to_datetime(df['Datum'], format='%Y-%m-%d', errors = 'coerce')
df_temp = df[df['Datum2'].isna()]
df.loc[df_temp.index, 'Datum']= '2018-01-01'
df['Datum'] = pd.to_datetime(df['Datum'])
df['Jaar'] = df['Datum'].dt.year
df.drop('Datum2', axis=1, inplace=True)

In [ ]:
df.head(10)

Filter NL

In [ ]:
len(df)

In [ ]:
df2 = df[(df['lat'] > 45) & (df['lat'] < 60) & (df['lon'] > 2) & (df['lon'] < 8)]
df2.head(10)

In [ ]:
len(df2)

In [ ]:
df2 = df2[df2['Ongeldig']==False]
len(df2)

## Waterschappen dataset

In [ ]:
waterschappen = gpd.read_file('../.data/in/waterschapsgebieden.json')
water_nl = waterschappen.to_crs(epsg=4326)

In [ ]:
f,ax = plt.subplots(1,1, figsize=(10, 8), sharex=True, sharey=True, dpi=140)
plt.title('waterschappen')
divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="3%",pad=0,alpha=0.5)
waterschappen.plot('text', ax=ax, alpha=0.5, cmap='viridis', edgecolor='k', legend=False, linewidth=0.1)
plt.show()

# Waarnemingen in waterschapsgebieden

In [ ]:
size = (30,24)
gdf = gpd.GeoDataFrame(
    df2, crs='epsg:4326', geometry=gpd.points_from_xy(df2.lon, df2.lat))

ax = waterschappen.to_crs(epsg=4326).plot(figsize=size, alpha=1, edgecolor='#fff', color='#555')
cx.add_basemap(ax, crs=water_nl.crs, source=cx.providers.CartoDB.Positron)
gdf.plot(column='Cijfer waterkwaliteit', ax=ax, cmap='RdYlGn', figsize=size, legend=True)

# Merge waarnemingen met waterschapsgebieden

In [ ]:
waterschappen = waterschappen.to_crs(epsg=4326)
gdf = gpd.GeoDataFrame(
    df2, crs='epsg:4326', geometry=gpd.points_from_xy(df2.lon, df2.lat))

within_waterschappen = gpd.sjoin(gdf, waterschappen, how="inner", predicate='within')

within_waterschappen.head(10)

# Waterkwaliteit in waterschapsgebieden

In [ ]:
stats = within_waterschappen.groupby('text').agg({'Cijfer waterkwaliteit':'mean'})
result = pd.merge(waterschappen, stats, left_on='text',right_index=True,how='outer')

## Map

In [ ]:
colors = ["#f64242","#f95444","#fc6646", "#ff7949", "#ffa530", "#ffd218", "#fff500", "#baf222", "#75e645", "#31da68"]
vmin = 0
vmax = 10
vstep = 1
nums = np.arange(vmin,vmax + vstep,vstep)
colormap2 = StepColormap(colors,nums,vmin,vmax, 'Indicatie waterkwaliteit')
result.explore(column='Cijfer waterkwaliteit', tooltip=['Cijfer waterkwaliteit', 'text'], tiles=None, cmap=colormap2, control_scale=True, style_kwds=dict(fillOpacity=1, color="#666", weight=.5))

# Waarnemingen in waterschapsgebieden

In [ ]:
countscumulative = within_waterschappen.groupby(['text']).agg({'Meting ID':'count'})
countscumulative.sort_values(by='Meting ID', ascending=False)

output = countscumulative.apply(lambda x:pd.cut(x, bins=[0,50,100,200,500,1000]))
output = countscumulative.join(output.add_prefix('bin_'))

result = pd.merge(waterschappen, output, on='text', how='outer')

ax = result.plot(column='bin_Meting ID', cmap='magma_r', figsize=(20,16), legend=True, edgecolor='#444')
ax.get_legend().set_bbox_to_anchor((1.4, 1))
ax.set(facecolor = "#dddddd")

# Waarnemingen heatmap

## Cumulatief

In [ ]:
# alle jaargangen bij elkaar
heatmap = folium.Map(location = [52, 5], tiles='Cartodb positron', zoom_start = 7)
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in within_waterschappen.geometry ]
plugins.HeatMap(heat_data, radius=8, blur=4).add_to(heatmap)

heatmap

## 2018

In [ ]:
jaar = 2018
heatmap = folium.Map(location = [52, 5], tiles='Cartodb positron', zoom_start = 7)
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in within_waterschappen[within_waterschappen['Jaar']==jaar].geometry ]
plugins.HeatMap(heat_data, radius=8, blur=4).add_to(heatmap)

heatmap

## 2019

In [ ]:
jaar = 2019
heatmap = folium.Map(location = [52, 5], tiles='Cartodb positron', zoom_start = 7)
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in within_waterschappen[within_waterschappen['Jaar']==jaar].geometry ]
plugins.HeatMap(heat_data, radius=8, blur=4).add_to(heatmap)

heatmap

## 2020

In [ ]:
jaar = 2020
heatmap = folium.Map(location = [52, 5], tiles='Cartodb positron', zoom_start = 7)
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in within_waterschappen[within_waterschappen['Jaar']==jaar].geometry ]
plugins.HeatMap(heat_data, radius=8, blur=4).add_to(heatmap)

heatmap

## 2021

In [ ]:
jaar = 2021
heatmap = folium.Map(location = [52, 5], tiles='Cartodb positron', zoom_start = 7)
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in within_waterschappen[within_waterschappen['Jaar']==jaar].geometry ]
plugins.HeatMap(heat_data, radius=8, blur=4).add_to(heatmap)

heatmap

# Dataset opslaan

In [ ]:
# inclusief:
# - waterschappen (text-kolom)
# - jaar-kolom

within_waterschappen.to_csv('../.data/out/waterdiertjes_en_waterschappen.csv', index=True)